# 异步计算
:label:`sec_async`

今天的计算机是高度并行的系统，包含多个CPU核心（通常每个核心有多个线程）、每个GPU上的多个处理单元，以及每台设备上通常有多个GPU。简而言之，我们可以同时处理许多不同的事情，通常是在不同的设备上。不幸的是，Python并不是编写并行和异步代码的好方法，至少在没有一些额外帮助的情况下不是。毕竟，Python是单线程的，而且这种情况在未来不太可能改变。深度学习框架如MXNet和TensorFlow采用了*异步编程*模型来提高性能，
而PyTorch则使用了Python自己的调度器，导致了不同的性能权衡。
对于PyTorch，默认情况下，GPU操作是异步的。当你调用一个使用GPU的函数时，这些操作会被加入到特定设备的队列中，但不一定立即执行。这使我们能够并行执行更多的计算，包括在CPU或其他GPU上的操作。

因此，了解异步编程的工作原理有助于我们通过主动减少计算需求和相互依赖性来开发更高效的程序。这使我们能够减少内存开销并提高处理器利用率。

In [1]:
import os
import subprocess
import numpy
import torch
from torch import nn
from d2l import torch as d2l

## 异步通过后端

为了热身，考虑以下这个简单的示例问题：我们想要生成一个随机矩阵并对其进行乘法运算。让我们分别在NumPy和PyTorch张量中执行此操作，以查看差异。
请注意，PyTorch `tensor`是在GPU上定义的。

In [2]:
# Warmup for GPU computation
device = d2l.try_gpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 1.4693 sec
torch: 0.0022 sec


通过PyTorch得到的基准输出快了几个数量级。
NumPy点积是在CPU处理器上执行的，而
PyTorch矩阵乘法是在GPU上执行的，因此后者
预计会快得多。但巨大的时间差异表明还有其他原因。
默认情况下，PyTorch中的GPU操作是异步的。
强制PyTorch在返回之前完成所有计算显示了
之前发生的情况：后端正在执行计算
而前端将控制权返回给Python。

In [3]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.0058 sec


总体而言，PyTorch 有一个前端用于与用户直接交互，例如通过 Python，以及一个由系统用来执行计算的后端。
如 :numref:`fig_frontends` 所示，用户可以用各种前端语言编写 PyTorch 程序，如 Python 和 C++。无论使用哪种前端编程语言，PyTorch 程序的执行主要发生在 C++ 实现的后端。前端语言发出的操作会被传递到后端进行执行。
后端管理自己的线程，这些线程持续收集并执行排队的任务。
请注意，为了使这一切正常工作，后端必须能够跟踪计算图中各个步骤之间的依赖关系。
因此，无法并行化相互依赖的操作。

![编程语言前端和深度学习框架后端。](../img/frontends.png)
:width:`300px`
:label:`fig_frontends`

让我们再看一个玩具示例来更好地理解依赖图。

In [4]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='cuda:0')

![后端跟踪计算图中各个步骤之间的依赖关系。](../img/asyncgraph.svg)
:label:`fig_asyncgraph`



上面的代码片段也在 :numref:`fig_asyncgraph` 中进行了说明。每当 Python 前端线程执行前三条语句中的任何一条时，它只是将任务返回到后端队列。当需要*打印*最后一条语句的结果时，Python 前端线程将等待 C++ 后端线程完成变量 `z` 的结果计算。这种设计的一个好处是 Python 前端线程不需要执行实际的计算。因此，无论 Python 的性能如何，对程序的整体性能影响很小。:numref:`fig_threading` 说明了前端和后端是如何交互的。

![前端和后端的交互。](../img/threading.svg)
:label:`fig_threading`




## 障碍和阻塞器

## 提高计算能力

## 摘要

* 深度学习框架可以将Python前端与执行后端解耦。这允许快速异步地向后端插入命令并实现相关的并行性。
* 异步性使得前端响应非常迅速。但是，请注意不要过度填充任务队列，因为这可能导致内存消耗过大。建议每个小批量同步一次，以保持前端和后端大致同步。
* 芯片供应商提供了先进的性能分析工具，可以更细致地了解深度学习的效率。

## 练习题

1. 在CPU上，对本节中的相同矩阵乘法操作进行基准测试。您是否仍能通过后端观察到异步性？

[讨论](https://discuss.d2l.ai/t/2564)